In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
normal_data=pd.read_csv("../Normal_data.csv")
metasploti_data=pd.read_csv("../metasploitable-2.csv")
ovs_data=pd.read_csv("../OVS.csv")

In [3]:
frames=[normal_data,metasploti_data,ovs_data]
data=pd.concat(frames,ignore_index=True)

In [4]:
data.Label = data.Label.str.replace(' ', '')
print(data.Label.unique())

['Normal' 'U2R' 'BFA' 'DDoS' 'DoS' 'Probe' 'Web-Attack' 'BOTNET']


In [5]:
data=data.loc[:, (data != 0).any(axis=0)]

In [6]:
data3 = data.loc[:, data.columns.difference(['Flow ID', 'Src IP','Dst IP','Timestamp','Src Port','Dst Port','Protocol'])]
data3=data3.sample(frac=1, replace=True, random_state=3).reset_index(drop=True)
data3.head()

,ACK Flag Cnt,Active Max,Active Mean,Active Min,Active Std,Bwd Header Len,Bwd IAT Max,Bwd IAT Mean,Bwd IAT Min,Bwd IAT Std,...,SYN Flag Cnt,Subflow Bwd Byts,Subflow Bwd Pkts,Subflow Fwd Byts,Subflow Fwd Pkts,Tot Bwd Pkts,Tot Fwd Pkts,TotLen Bwd Pkts,TotLen Fwd Pkts,URG Flag Cnt
0,0,0.0,0.0,0.0,0.0,0,12.0,12.0,12.0,0.00000,...,0,0,2,0,0,2,0,0.0,0.0,0
1,1,0.0,0.0,0.0,0.0,64,3369.0,3369.0,3369.0,0.00000,...,0,0,2,0,0,2,0,0.0,0.0,0
2,0,0.0,0.0,0.0,0.0,0,13.0,13.0,13.0,0.00000,...,0,0,2,0,0,2,0,0.0,0.0,0
3,1,0.0,0.0,0.0,0.0,20,0.0,0.0,0.0,0.00000,...,0,0,1,248,2,1,2,0.0,248.0,0
4,0,0.0,0.0,0.0,0.0,144,167136.0,115976.2,2573.0,72264.35192,...,1,3510,6,370,7,6,7,3510.0,370.0,0


In [7]:
data3.Label.replace(['U2R','BFA','DDoS','DoS','Probe','Web-Attack','BOTNET'],'Saldiri',inplace=True)
data3.Label.replace("Saldiri","1",inplace=True)
data3.Label.replace("Normal","0",inplace=True)
data3.Label.unique()

array(['1', '0'], dtype=object)

In [8]:
X=data3[["ACK Flag Cnt",
"Active Max",
"Active Mean",
"Active Min",
"Active Std",
"Bwd Header Len",
"Bwd IAT Max",
"Bwd IAT Mean",
"Bwd IAT Min",
"Bwd IAT Std",
"Bwd IAT Tot",
"Bwd PSH Flags",
"Bwd Pkt Len Max",
"Bwd Pkt Len Mean",
"Bwd Pkt Len Min",
"Bwd Pkt Len Std",
"Bwd Pkts/s",
"Bwd URG Flags",
"Down/Up Ratio",
"FIN Flag Cnt",
"Flow Byts/s",
"Flow Duration",
"Flow IAT Max",
"Flow IAT Mean",
"Flow IAT Min",
"Flow IAT Std",
"Flow Pkts/s",
"Fwd Act Data Pkts",
"Fwd Header Len",
"Fwd IAT Max",
"Fwd IAT Mean",
"Fwd IAT Min",
"Fwd IAT Std",
"Fwd IAT Tot",
"Fwd Pkt Len Max",
"Fwd Pkt Len Mean",
"Fwd Pkt Len Min",
"Fwd Pkt Len Std",
"Fwd Seg Size Avg",
"Idle Mean",
"Idle Min",
"Idle Std",
"Init Bwd Win Byts",
"PSH Flag Cnt",
"Pkt Len Max",
"Pkt Len Mean",
"Pkt Len Min",
"Pkt Len Std",
"Pkt Len Var",
"Pkt Size Avg",
"RST Flag Cnt",
"SYN Flag Cnt",
"Subflow Bwd Byts",
"Subflow Bwd Pkts",
"Subflow Fwd Pkts",
"Tot Bwd Pkts",
"Tot Fwd Pkts",
"TotLen Bwd Pkts",
"URG Flag Cnt"]]
y=data3.Label
print(X.shape)
print(y.shape)

(343889, 59)
(343889,)


In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-0.61566854, -0.08817253, -0.07895971, ..., -0.06209157,
        -0.02986745, -0.10935765],
       [ 1.62425061, -0.08817253, -0.07895971, ..., -0.06209157,
        -0.02986745, -0.10935765],
       [-0.61566854, -0.08817253, -0.07895971, ..., -0.06209157,
        -0.02986745, -0.10935765],
       ...,
       [ 1.62425061, -0.08817253, -0.07895971, ..., -0.06209157,
        -0.02986745, -0.10935765],
       [ 1.62425061, -0.08709042, -0.07730406, ..., -0.00703453,
        -0.02974927, -0.10935765],
       [ 1.62425061, -0.08817253, -0.07895971, ..., -0.06209157,
        -0.02986745, -0.10935765]])

In [10]:
y= pd.get_dummies(y, drop_first=True)
X_train, X_test, y_train, y_test = X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, test_size=0.25, random_state=1)

In [11]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

C:\Users\Birol\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9966384795226408

In [12]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
model = XGBRegressor(n_estimators=1000,learning_rate=0.5,n_jobs=4)
model.fit(X_train, y_train, 
             early_stopping_rounds=500, 
             eval_set=[(X_test, y_test)],
             verbose=False)
pred=model.predict(X_test)
print(model.score(X_test,y_test))
print(mean_absolute_error(y_test,pred))

C:\Users\Birol\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.9989439831686646
0.00031474332366912593


In [13]:
from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor(random_state=0)
model.fit(X_train,y_train)
pred=model.predict(X_test)
print(model.score(X_test,y_test))
print(mean_absolute_error(y_test,pred))

0.9993469316241241
0.00031474332366912593


In [14]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=3,weights='uniform')
model.fit(X_train,y_train)
pred=model.predict(X_test)

print(model.score(X_test,y_test))
print(mean_absolute_error(y_test,pred))

0.9998022635013318
0.00031474332366912593


In [12]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(random_state=1)
model.fit(X_train,y_train)
pred=model.predict(X_test)

print("----- Random Forest ------")
print(model.score(X_test,y_test))
print(mean_absolute_error(y_test,pred))
print("----- Random Forest ------")

----- Random Forest ------
0.9993258084903222


NameError: name 'mean_absolute_error' is not defined

In [13]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
model.fit(X_train,y_train)
pred=model.predict(X_test)

print(model.score(X_test,y_test))
print(mean_absolute_error(y_test,pred))

0.9992206855640724


NameError: name 'mean_absolute_error' is not defined

In [11]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)
pred=model.predict(X_test)
print("----- GaussianNB ------")
print(model.score(X_test,y_test))
print(mean_absolute_error(y_test,pred))
print("----- GaussianNB ------")

----- GaussianNB ------
0.9388180010003141


NameError: name 'mean_absolute_error' is not defined

In [ ]:
from sklearn.svm import SVC
model = SVC(C=10.0, random_state=1, kernel='linear')
model.fit(X_train,y_train)
pred=model.predict(X_test)

print(model.score(X_test,y_test))
print(mean_absolute_error(y_test,pred))

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.layers import  Embedding, SimpleRNN, LSTM,Masking,Bidirectional
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import model_from_json,model_from_yaml,load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence,text
from sklearn.preprocessing import LabelEncoder
from keras import metrics, regularizers
from keras.optimizers import RMSprop,Adam
from keras import backend as K

In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape) 
print(y_test.shape)

(257916, 59)
(85973, 59)
(257916, 1)
(85973, 1)


In [13]:
# Model için gerekli değerler
#epochs_sayisi=5
batch_size=512
# Çıktı gözükmemesi için
verbose=1
validation_split=0.1
max_len=len(X.columns)
vocab_size=10000

In [14]:
def rnn(epoch_sayisi):
    model=Sequential()
    model.add(Embedding(vocab_size,max_len, trainable=True,input_length=max_len))
    model.add(SimpleRNN(40,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(32,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(24,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(16,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(4,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001), metrics=['acc'])
    history=model.fit(X_train, y_train, epochs=epoch_sayisi, batch_size=batch_size,
                      verbose=verbose,validation_split=validation_split)
    return history,model

In [15]:
def lstm(epoch_sayisi):
    model=Sequential()
    model.add(Embedding(vocab_size,max_len, trainable=True,input_length=max_len))
    model.add(LSTM(32,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(16,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(4,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])
    history=model.fit(X_train, y_train, epochs=epoch_sayisi, batch_size=batch_size,
                      verbose=verbose,validation_split=validation_split)
    return history,model

In [16]:
def bilstm(epoch_sayisi):
    model=Sequential()
    model.add(Embedding(vocab_size,max_len, trainable=True,input_length=max_len))
    model.add(Bidirectional(LSTM(32,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(16,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(4,activation='relu',kernel_regularizer=regularizers.l2(0.01))))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])
    history=model.fit(X_train, y_train, epochs=epoch_sayisi, batch_size=batch_size,
                      verbose=verbose,validation_split=validation_split)
    return history,model

In [17]:
def cnn(epoch_sayisi):
    model=Sequential()
    model.add(Embedding(vocab_size,max_len, trainable=True,input_length=max_len))
    model.add(Conv1D(32,1,kernel_regularizer=regularizers.l2(0.01),activation='relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(16,1,kernel_regularizer=regularizers.l2(0.01),activation='relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(4,1,kernel_regularizer=regularizers.l2(0.01),activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])
    history=model.fit(X_train, y_train, epochs=epoch_sayisi, batch_size=batch_size,
                      verbose=verbose,validation_split=validation_split)
    return history,model
    #model.summary()

In [20]:
#epoch_sayisi=int(input("Eposh sayisini giriniz..:"))

print('''
    Uygulanacak algoritmayı seçiniz:
    1 - CNN
    2 - RNN
    3 - LSTM
    ''')

epoch_sayisi=2
for algoritma in ["rnn" ,"cnn", "lstm"]:
    print(f"\n\n {algoritma} ---- ")
    #algoritma=input("Seçim:").lower()
    if(algoritma=="cnn"):
        history,model=cnn(epoch_sayisi)
    elif(algoritma=="rnn"):
        history,model=rnn(epoch_sayisi)
    elif(algoritma=="lstm"):
        history,model=lstm(epoch_sayisi)

#print("**** %s modelinde %s epochluk Acc %.2f  Loss %.2f" %(algoritma,epoch_sayisi,history.history['acc'][epoch_sayisi-1]*100,history.history['loss'][epoch_sayisi-1]*100))

#rnn_test=test(algoritma,model)


    Uygulanacak algoritmayı seçiniz:
    1 - CNN
    2 - RNN
    3 - LSTM
    


 rnn ---- 
Epoch 1/2


C:\Users\Birol\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


454/454 [==============================] - 241s 503ms/step - loss: 0.5443 - acc: 0.9471 - val_loss: 0.1388 - val_acc: 0.9935
Epoch 2/2
454/454 [==============================] - 169s 373ms/step - loss: 0.1131 - acc: 0.9893 - val_loss: 0.0753 - val_acc: 0.9906


 cnn ---- 
Epoch 1/2


C:\Users\Birol\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


454/454 [==============================] - 23s 47ms/step - loss: 0.6269 - acc: 0.8008 - val_loss: 0.4997 - val_acc: 0.8020
Epoch 2/2
454/454 [==============================] - 19s 42ms/step - loss: 0.4997 - acc: 0.8009 - val_loss: 0.4986 - val_acc: 0.8020


 lstm ---- 
Epoch 1/2


C:\Users\Birol\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


454/454 [==============================] - 132s 285ms/step - loss: 0.6388 - acc: 0.8634 - val_loss: 0.3047 - val_acc: 0.9594
Epoch 2/2
454/454 [==============================] - 147s 324ms/step - loss: 1.6188 - acc: 0.9468 - val_loss: 0.2515 - val_acc: 0.9568
